## Merging DataFrames
Inthis notebook we're going to address how you can bring multiple dataframe objects together, either by merging them horizontally, or by concatenating them vertically. Before we jump into the code, we need to address a little relational theory and to get some language convetions down. A Venn Diagram is traditionally used to show set membership. For example, the circle on the left is the population of students at a university. The circle on the right is the population of staff at university. And the overlapping regio un this middle are all of those students who are also staff. Maybe these students run tutorials for a course, or grade assignments, or engage in running research experiments. So, this diagram shows two populations whom we nught have data about, but there is overlap between those populations.
When it comes to translation this pandas, we can think of the case where we might have these two populations as indices in separate DataFrames, maybe with the label of PersonName. When we want to join the DataFrames together, we have some choices to maje, First what if we want a listo of all the people regardless of whether they're staff or student, and all of the inforamtion we can get of them? In database terminology, this is called a full outer join. And in ser theory, it's called a union. In the Venn diagram, it represents everyone in any circle.
FULL OUTER JOIN (UNION) All elements of both sets.
It's quite possible though that we only want those people who we hace maximum information for, these people who are both staff and students. Maybe being a staff member and a studen involves getting a tuition waiver, and we want to calculate the cost of this. In database terminology, this is called an inner join. Or in set theory, the intersection.
INNER JOIN (INTERSECTION) Only elments are in both sets.

In [2]:
#Let's to create two dataframes for explore the diferents forms to merging it.
import pandas as pd

staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df.set_index('Name', inplace = True)

student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])

student_df.set_index('Name', inplace = True)

print(staff_df)
print(student_df)
# There's some overlap in these DataFrames, in that James and Sally are both students and staff, but Mike and Kelly are not. Importantly, both DataFrames are indexed along the value we want to
# merge them on, which is called Name.

                 Role
Name                 
Kelly  Director of HR
Sally  Course liasion
James          Grader
            School
Name              
James     Business
Mike           Law
Sally  Engineering


In [3]:
# If we want the union of these, we would call merge() passing in the DataFrame on the left and the DataFrame on the right and telling merge that we want it to use an outer join.
# We want to use the left and right indices as the joining columns.
pd.merge(staff_df, student_df, how = 'outer', left_index = True, right_index = True) # left_index and reight index are for use both indexes to join dataframes

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


In [8]:
# We see in the resulting DataFrame that everyone is listed. And since Mike dos not have a role, and John does not have a school those cells are listed as missing values.
# If we wanted to ge the intersection, that is, just those who are a student AND a staff, we could ser the how attribute to inner. Again, we set both left and right indices to be true as the joining cols.
pd.merge(staff_df, student_df, how = 'inner', left_index = True, right_index = True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


In [9]:
# And we see the resulting DataFrame has only James and Sally in it. Now there are two other common use cases whe merging DataFrames, and both are examples of what we would call set addition. 
# The first is when we would want to get a list of all staff regardless of wether they were students or not. Buy if ther were students, we would want to get their student details as well. To do this
# we would use a left join. It is important to note the order of dataframes in this function: the first dataframe is the left dataframe and the second is the right.
pd.merge(staff_df, student_df, how = 'left', left_index = True, right_index = True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


In [10]:
# We could probably guess what comes next. We want a list of all of the students and their roles if they were also staff. To do this we would do a right join.
pd.merge(staff_df, student_df, how = 'right', left_index = True, right_index = True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


In [11]:
# We can also do it another way. The merge method has a couple of other interesting parameters. First, you don't need to use indeces to join on, you can use columns as well. Here's an example.
# Here we have a parameter called "on", and we can assign a column that both dataframe has the joining column.
# First, lets remove our index from both of our dataframes
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

# Now lets merge using the on parameter
pd.merge(staff_df, student_df, how='right', on='Name')

,Name,Role,School
0,James,Grader,Business
1,Mike,NaN,Law
2,Sally,Course liasion,Engineering


In [12]:
# Using the "on" parameter instead of a the index is how I find myself usign merge() the most. So what happens when we have conflicts between the DataFrames? Let's try take a look by creating new staff 
# and student DataFrames that have a location information added to them.
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}])
# In this staff DataFrame, this is an office location where we can find the staff person. And we can see the Director of HR is on Satate Street, while the two srtudents are on Washington Avenue, and
# these locations just happen to be right outside my window as I film this. But for the studen DataFrame , the location information is actually their home address.
# Ther merge function preserves this informations, but appends and _x or _y to help differentiate betwwen which index went which column of data. The _x is always the left DataFrame information, and
# the _y is always the right DataFrame information.
# Here, if we want all the staff information regardless of whether they were students or not. But if they were students, we could want to get their student details as well. Then we can do a left join
# and on the column of Name
pd.merge(staff_df, student_df, how = 'left', on = 'Name')
# From the outpout, we can see there are columns locations_x and location_y. Location x refers to the location column in the left dataframe which is staff dataframe and location_y refers to the location
#column in the right dataFrame, which is studen dataFrame.

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


In [13]:
# Before we leave merging DataFrames, Let's talk about multi - indexing and mulple columns. In's quite possible that the first name for students and staff might overla but the last name might not. 
# In his case, we use a list of the multiple columns that should be used to join keys from both dataframes on the on parameter. Recall that the column name (s) assigned to the on parameter needs
# to exist in both dataframes.
# Here's an example with some new student and staff data
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])
# AS you see here, James Wilde and James Hammond don't match on both keys since they have different last names. So we could expect that an inner join doesn't include these individuals in the output
# an only Sally Brooks will be retained
pd.merge(staff_df, student_df, how = 'inner', on = ['First Name', 'Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering
